# <center> Soluciones Numéricas a la Ecuación de Calor Aplicadas a la Ecuación de Black-Scholes-Merton 2D en CUDA Python </center>

<center> Faculta de Ciencias </center>
<center> Universidad Nacional Autónoma de México </center>
<center>Temas Selectos de Análisis Numérico 2026-1</center>

$$\frac{\partial u}{\partial t} = \alpha \left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right)$$

$$\frac{\partial V}{\partial t} + \frac{1}{2}\sigma^2S^2\frac{\partial^2 V}{\partial S^2} + rS\frac{\partial V}{\partial S}-rV = 0$$

<center> Santiago Romero (santiago.rp@ciencias.unam.mx) </center>

## Preámbulo

### Paquetería

**Únicamente si estás en Google Colab**, sube el archivo `requirements.txt` a
tu entorno de Google Colab presionando el símbolo de folder a la izquierda y
subiéndolo ahí. Después corre la siguiente celda, tras borrar el '#'.

In [2]:
# !pip install requirements.txt

Importamos las bibliotecas y módulos necesarios

In [80]:
import numpy as np
import math
from numba import cuda, jit, vectorize, float64
import cupy as cp

### Pruebas de entorno

Es importante para correr el *notebook* que las siguientes dos celdas se ejecuten exitosamente. De lo contrario nos hace falta estar en un ambiente de CUDA apropiado. De ser el caso, revisa el README dentro del directorio de `codigo/`.

In [4]:
print("CUDA disponible:", cuda.is_available())

if cuda.is_available():
    device = cuda.get_current_device()
    print("GPU detectado:", device.name)
    print("Capacidad de cómputo:", device.compute_capability)
else:
    print("❌ ERROR: No se pudo acceder a la GPU.")

CUDA disponible: True
GPU detectado: b'NVIDIA GeForce GTX 1060 6GB'
Capacidad de cómputo: (6, 1)


In [5]:
assert(cuda.is_available())

# Kernel de calentamiento y prueba
@cuda.jit
def add_kernel(a, b, out):
    idx = cuda.grid(1)
    if idx < out.size:
        out[idx] = a[idx] + b[idx]
# Datos de prueba
n = 1_000_000
a = np.ones(n, dtype=np.float32)
b = np.ones(n, dtype=np.float32)
out = np.zeros(n, dtype=np.float32)
# Copia a la GPU
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_out = cuda.to_device(out)
# Lanzamiento de kernel
threads_per_block = 256
blocks_per_grid = (n + threads_per_block - 1) // threads_per_block
add_kernel[blocks_per_grid, threads_per_block](d_a, d_b, d_out)
# Guardamos resultado
result = d_out.copy_to_host()
# Validamos
if np.all(result == 2):
    print("✅ ÉXITO: La GPU esta lista para ser usada")
else:
    print("❌ ERROR: La GPU tiene algún problema.")

✅ ÉXITO: La GPU esta lista para ser usada


## Definición de un problema BSM-2D

### Función de *payoff* $\varphi_K$

Definamos nuestra función $\varphi_K : \mathbb{R}^2 \to \mathbb{R}$ como un *european basket call* ponderado con $\varphi_K(S_1,S_2) = \max\left\{0.65S_1 + 0.35S_2 - K, 0\right\}$

In [6]:
def _payoff(s1:np.float64, s2:np.float64, k:np.float64) -> np.float64:
    return max(0.65*s1 + 0.35*s2 - k, 0)
h_payoff = jit(nopython=True)(_payoff) # Versión rápida en CPU
d_payoff = cuda.jit(device=True)(_payoff) # Versión en GPU

### Matriz L y su inversa

Definamos las funciones de transformación de un problema BSM a calor y viceversa

In [7]:
@jit(nopython=True)
def make_L(rho, s1, s2):
    """
    Entrada:
        rho - rho
        s1 - sigma_1
        s2 - sigma_2
    Salida:
        L
        L_inv
    """
    assert abs(rho) < 1 - 1e-6, "Queremos |rho| < 1"
    assert s1 >= 0, "sigma 1 debe ser no-negativo"
    assert s2 >= 0, "sigma 2 debe ser no-negativo"
    if abs(rho)<1e-8:
        L = np.array([[1.0/s1,0.0],[0.0,1.0/s2]], dtype=np.float64)
        L_inv = np.array([[s1,0.0],[0.0,s2]], dtype=np.float64)
        return L, L_inv
    s12 = s1*s1
    s22 = s2*s2
    rs1s2 = rho*s1*s2
    rs1s2sq = rs1s2**2
    lhs = 0.5*(s12 + s22)
    rhs = 0.5*np.sqrt((s12-s22)**2 + 4*rs1s2sq)
    lam1 = lhs-rhs
    lam2 = lhs+rhs
    scale1 = 1.0/np.sqrt(rs1s2sq + (s12 - lam1)**2)
    scale2 = 1.0/np.sqrt(rs1s2sq + (s12 - lam2)**2)
    v1 = scale1*np.array([-rs1s2, s12-lam1], dtype=np.float64)
    v2 = scale2*np.array([-rs1s2, s12-lam2], dtype=np.float64)
    f1 = 1.0/np.sqrt(lam1)
    f2 = 1.0/np.sqrt(lam2)
    L = f1*np.outer(v1,v1) + f2*np.outer(v2,v2)
    L_det = L[0,0]*L[1,1] - L[1,0]*L[0,1]
    L_inv = (1.0/L_det)*np.array([[L[1,1],-L[0,1]],[-L[1,0],L[0,0]]], dtype=np.float64)
    return L, L_inv

### Transformación $S\to z$

In [8]:
# Regresamos el vector z a partir de las condiciones del problema y coordenadas en S-espacio
def to_z(S, s1, s2, r, t, T, L):
    """
    Entrada
        S - vector S_1, S_2
        s1 - sigma_1
        s2 - sigma_2
        r - r
        t - tiempo actual
        T - tiempo final
        L - L
    Salida
        - vector z
        - tiempo tau
    """
    tau = T - t
    y1 = np.ln(S[0]) + (r-s1*s1*0.5)*t
    y2 = np.ln(S[1]) + (r-s2*s2*0.5)*t
    y = np.array([y1,y2], dtype=np.float64)
    z = np.dot(L,y)
    return z, tau

### Transformación $z\to S$

In [9]:
# Obtenemos el valor de S en un z particular
def to_s(z, s1, s2, r, tau, T, L_inv):
    """
    Entrada
        z - vector z_1, z_2
        s1 - sigma_1
        s2 - sigma_2
        r - r
        tau - tiempo tau
        T - tiempo final
        L_inv - inversa de L
    Salida
        - vector S
        - tiempo t
    """
    t = T - tau
    rhs1 = t*(r - 0.5*s1*s1)
    rhs2 = t*(r - 0.5*s2*s2)
    on_exp = np.dot(L_inv, z) + np.array([rhs1,rhs2], dtype=np.float64)
    S = np.exp(on_exp)
    return S

### Valor inicial de u

In [10]:
# Llena la malla inicial en GPU como una función universal (ufunc)
@vectorize([
    'float64(float64, float64, float64, float64, float64, float64, float64, '
    'float64, float64, float64, float64, float64, float64)'
], target='cuda')
def initial_u(z1, z2, maxz, threshold, s1, s2, r, k, T, L_inv00, L_inv01, L_inv10, L_inv11):
    """
    
    """
    rhs1 = T * (r - 0.5*s1*s1)
    rhs2 = T * (r - 0.5*s2*s2)
    on_exp0 = L_inv00 * z1 + L_inv01 * z2 + rhs1
    on_exp1 = L_inv10 * z1 + L_inv11 * z2 + rhs2
    S1 = math.exp(on_exp0)
    S2 = math.exp(on_exp1)
    payoff_val = h_payoff(S1, S2, k)
    val = math.exp(r * T) * payoff_val / (S1 + S2)
    bnd = max(abs(z1), abs(z2)) >= maxz - 1e-12
    if bnd and S1 + S2 >= threshold: # frontera especial
        val /= S1 + S2
        return val if val > 0.0 else 0.0
    else: # nodo interno o frontera común
        return val if val > 0.0 or bnd else 0.0

### Discretización de la malla

In [11]:
def make_mesh(width, interest, threshold, s1, s2, r, k, T, L_inv):
    """
    Entrada
        width - el número de celdas en la discretización por cada eje (se guardará el impar siguiente)
        interest - el radio de interés de z alrededor del cero
        threshold - el umbral para determinar la condición de frontera
        s1 - sigma_1
        s2 - sigma_2
        T - tiempo final
    Salida
        - malla con los valores iniciales de u
        - valor máximo de z en cada coordenada
        - dz
    """
    width += 1 - (width % 2) # impar siguiente (o fijo si ya es impar)
    interest *= max(s1,s2,1)*max(1,math.sqrt(T)) # escalamiento (efectivamente zona fantasma)
    zlin1 = np.linspace(-interest, interest, width)
    zlin2 = np.linspace(-interest, interest, width)
    Z1, Z2 = np.meshgrid(zlin1, zlin2)
    # La malla inicial en GPU
    u0 = initial_u(Z1, Z2,
                   interest, threshold, s1, s2, r, k, T,
                   L_inv[0,0], L_inv[0,1], L_inv[1,0], L_inv[1,1])
    return u0, interest, zlin1[1] - zlin1[0]

In [12]:
# Malla de ejemplo
rho = 0.7
s1 = 0.2
s2 = 0.3
k = 0.01
r = 0.03
T = 2
width = 1000
zmax = 8
L, L_inv = make_L(rho, s1, s2)
make_mesh(width, zmax, 50, s1, s2, r, k, T, L_inv)

(array([[0.45020833, 0.45092789, 0.45164525, ..., 0.67461647, 0.67465391,
         0.67469125],
        [0.45047911, 0.45119684, 0.45191238, ..., 0.67457415, 0.67461165,
         0.67464905],
        [0.45074831, 0.45146422, 0.45217794, ..., 0.6745316 , 0.67456916,
         0.67460662],
        ...,
        [0.38357851, 0.383606  , 0.38363353, ..., 0.44736886, 0.44748423,
         0.00502585],
        [0.38353032, 0.38355769, 0.38358511, ..., 0.44711402, 0.44722912,
         0.00499585],
        [0.38348231, 0.38350957, 0.38353687, ..., 0.00498851, 0.00497725,
         0.00496601]], shape=(1001, 1001)),
 11.313708498984761,
 np.float64(0.02262741699797033))

## Métodos Numéricos de Álgebra Lineal

### Gauss-Seidel Rojinegro (GPU)

Es posible implementar Gauss-Seidel como algoritmo de solución de sistemas lineales al dividir la matriz en nodos *negros* y *rojos* que actúan como barreras de sincronización para poder paralelizarlo. Si estamos en una *fase roja* sólo actualizamos rojos, y análogamente la *fase negra*.

Sería deseable implementar este método pero queda fuera del alcance del proyecto.

Se puede leer más en:
* [Fast GPU algorithms for implementing the red-black Gauss-Seidel method for Solving Partial Differential Equations](https://ieeexplore.ieee.org/document/6754958)
* [Development of Red-Black Gauss-Seidel Algorithm for
Efficiently Pricing Fixed Strike Asian Options based on
Arithmetic Average](https://ijettjournal.org/Volume-71/Issue-11/IJETT-V71I11P219.pdf)

### Gauss-Seidel+SOR (CPU)

Este es nuestro método de referencia en CPU

In [55]:
# Resuelve por sustitución la matriz triangular inferior
@jit(nopython=True)
def solve_triangular(L, b):
    n = L.shape[0]
    x = np.zeros_like(b)
    for i in range(n):
        s = 0.0
        for j in range(i):
            s += L[i, j] * x[j]
        x[i] = (b[i] - s) / L[i, i]
    return x

# Método de Gauss-Seidel con sobrerelajación (SOR)
@jit(nopython=True)
def gauss_seidel(A, b, x0, w=1.0, tol=1e-9, max_it=1000):
    n = A.shape[0]
    D = np.zeros_like(A)
    L = np.zeros_like(A)
    U = np.zeros_like(A)
    for i in range(n):
        D[i, i] = A[i, i]
        for j in range(i):
            L[i, j] = A[i, j]
        for j in range(i+1, n):
            U[i, j] = A[i, j]
    M = D/w + L
    N = (1/w - 1)*D - U

    x_prev = x0.copy()
    for k in range(max_it):
        x_new = solve_triangular(M, N.dot(x_prev) + b)
        if np.max(np.abs(x_new - x_prev)) <= tol:
            return x_new
        x_prev = x_new
    return x_prev

### Jacobi (GPU)

Aunque Jacobi toma más iteraciones en converger, es posible que paralelizarlo resulte en que termine mas rápido.

In [56]:
TILE = 256  # tamaño de mosaico en memoria compartida

# Realiza una iteración de Jacobi en la GPU
@cuda.jit
def jacobi_step(A, b, x_old, x_new, n):
    i = cuda.grid(1)
    if i >= n:
        return
    # x_old en memoria compartida
    sh_x = cuda.shared.array(shape=(TILE,), dtype=float64)
    acc = 0.0
    # Aplicamos el paso de Jacobi por mosaicos
    for t in range(0, n, TILE):
        # Cargamos el valor viejo
        j = t + cuda.threadIdx.x
        if j < n and cuda.threadIdx.x < TILE:
            sh_x[cuda.threadIdx.x] = x_old[j]
        cuda.syncthreads()
        # Acumulamos los valores útiles en el mosaico
        j_end = min(t + TILE, n)
        for jj in range(t, j_end):
            acc += A[i, jj] * sh_x[jj - t]
        cuda.syncthreads()
    # Terminamos y guardamos
    aii = A[i, i]
    xi_old = x_old[i]
    acc -= aii * xi_old
    x_new[i] = (b[i] - acc) / aii

# Calcula el residual, i.e. la distancia a la solución al cuadrado
@cuda.jit
def compute_residual(A, x, b, residual, n):
    i = cuda.grid(1)
    # Comparación parcial de Ax con b en memoria compartida
    val = 0.0
    if i < n:
        s = 0.0
        for j in range(n):
            s += A[i, j] * x[j]
        r_i = s - b[i]
        val = r_i * r_i
    sh = cuda.shared.array(shape=(256,), dtype=float64)
    tid = cuda.threadIdx.x
    sh[tid] = val
    cuda.syncthreads()
    # Reducción paralela para sumar las distancias cuadradas
    stride = cuda.blockDim.x // 2
    while stride > 0:
        if tid < stride:
            sh[tid] += sh[tid + stride]
        cuda.syncthreads()
        stride //= 2
    # Aumentamos el residual
    if tid == 0:
        cuda.atomic.add(residual, 0, sh[0])

# Desde CPU llama adecuadamente el método de Jacobi en GPU
def jacobi_cuda(h_A, h_b, x0=None, max_iters=500, tol=1e-8, debug=False):
    n = h_A.shape[0]
    assert h_A.shape == (n, n) # que la matriz sea cuadrada
    assert h_b.shape == (n,) # que el vector sea del tamaño correcto y efectivamente vector
    # Estimación inicial
    if x0 is None:
        h_x = np.zeros(n, dtype=np.float64)
    else:
        h_x = np.asarray(x0, dtype=np.float64)
    # Transferencia a GPU
    d_A = cuda.to_device(h_A)
    d_b = cuda.to_device(h_b)
    d_x_old = cuda.to_device(h_x)
    d_x_new = cuda.device_array_like(d_x_old)
    d_residual = cuda.to_device(np.array([0.0], dtype=np.float64))
    # Configuración de la kernel
    grid_size = (n + TILE - 1) // TILE
    # Iteraciones Jacobi
    for it in range(max_iters):
        jacobi_step[grid_size, TILE](d_A, d_b, d_x_old, d_x_new, n)
        d_residual.copy_to_device(np.array([0.0], dtype=np.float64))
        compute_residual[grid_size, TILE](d_A, d_x_new, d_b, d_residual, n)
        res2 = d_residual.copy_to_host()[0] # nota importante, aquí se sincroniza GPU con host
        res = math.sqrt(res2) # el residual real
        if debug == True or (debug != False and (it+1) % debug == 0): # mensajes de depuración
            print(f"iter {it+1}: ||Ax - b|| = {res:.6e}")
        if res < tol: # terminación temprana por tolerancia
            break
        d_x_old, d_x_new = d_x_new, d_x_old # swap de buffers viejo y actual
    return d_x_old.copy_to_host() # guardamos el resultado final

### Comparativa

In [76]:
np.random.seed(0)
n = 6000
A = np.random.randn(n, n)
acc = 0
for i in range(n):
    A[i, i] = np.sum(np.abs(A[i])) + 1.0
    acc += abs(A[i,i])
for i in range(n):
    A[i,i] += acc # forzando que sea muy diagonal-dominante
for i in range(n):
    for j in range(i):
        A[j,i] = A[i,j] # forzando simetría
b = np.random.randn(n)

In [77]:
%time JAC=jacobi_cuda(A, b, max_iters=1000, tol=1e-8, debug=100)

iter 100: ||Ax - b|| = 2.677305e-05
iter 200: ||Ax - b|| = 2.016792e-05
iter 300: ||Ax - b|| = 2.677306e-05
iter 400: ||Ax - b|| = 2.677305e-05
iter 500: ||Ax - b|| = 2.016792e-05
iter 600: ||Ax - b|| = 2.016792e-05
iter 700: ||Ax - b|| = 2.677306e-05
iter 800: ||Ax - b|| = 2.016792e-05
iter 900: ||Ax - b|| = 2.016792e-05
iter 1000: ||Ax - b|| = 2.677306e-05
CPU times: user 14.7 s, sys: 95.5 ms, total: 14.8 s
Wall time: 13.7 s


In [78]:
%time GSSOR=gauss_seidel(A,b,np.zeros_like(b))

CPU times: user 1.61 s, sys: 1.91 s, total: 3.52 s
Wall time: 2.01 s


In [79]:
%time NPLN=np.linalg.solve(A,b)

CPU times: user 5.86 s, sys: 263 ms, total: 6.12 s
Wall time: 5.66 s


In [81]:
%time CPLN=cp.linalg.solve(A,b)

CPU times: user 0 ns, sys: 8.09 ms, total: 8.09 ms
Wall time: 894 ms


ImportError: libcublas.so.13: cannot open shared object file: No such file or directory

In [75]:
print("Máxima diferencia en Jacobi vs NumPy",max(abs(NPLN-JAC)))
print("Máxima diferencia en SOR vs NumPy",max(abs(NPLN-GSSOR)))

Máxima diferencia en Jacobi vs NumPy 2.044035976589293e-13
Máxima diferencia en SOR vs NumPy 6.410815183408475e-19
